In [2]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize
from ratelimit import limits, sleep_and_retry
import requests

In [1]:
pip install ratelimit

  Created wheel for ratelimit: filename=ratelimit-2.2.1-cp37-none-any.whl size=5899 sha256=396af152e36e89c764f7805b399d23b412eb3fbb7ce2502c03428a14c1c1234b
  Stored in directory: C:\Users\sang\AppData\Local\pip\Cache\wheels\05\d9\82\3c6044cf1a54aab9151612458446d9b17a38416869e1b1d9b8
Successfully built ratelimit
Note: you may need to restart the kernel to use updated packages.


In [8]:
# https://pypi.org/project/ratelimit/

two_minutes = 120

@sleep_and_retry
@limits(calls=90, period=two_minutes)
def call_api(url):
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception('API response: {}'.format(response.status_code))
    return response


In [9]:
# Global Variables
KEY = 'RGAPI-b1d840c5-c623-4af8-b0a2-9500fdbe24fb'
API_BASE = 'https://na1.api.riotgames.com/'
API_BASE_REGION = 'https://americas.api.riotgames.com/'
 
TIERS = ['IRON' , 'BRONZE' , 'SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']
DIVISIONS = ['I' , 'II', 'III', 'IV']
TIERS_LIST = []
for tier in TIERS:
  for division in DIVISIONS:
    TIER_DIVISION = tier + '/' + division
    TIERS_LIST.append(TIER_DIVISION)

TIER_URLS = []
for tiers_divis in TIERS_LIST: 
  url = API_BASE + 'tft/league/v1/entries/' + tiers_divis + '?page=1&api_key=' + KEY
  TIER_URLS.append(url)

In [10]:
TIER_URLS

['https://na1.api.riotgames.com/tft/league/v1/entries/IRON/I?page=1&api_key=RGAPI-b1d840c5-c623-4af8-b0a2-9500fdbe24fb',
 'https://na1.api.riotgames.com/tft/league/v1/entries/BRONZE/I?page=1&api_key=RGAPI-b1d840c5-c623-4af8-b0a2-9500fdbe24fb']

In [11]:
# Get list of each summoner in each tier
tier_frames = []

for url in TIER_URLS:
  REQUEST_TIERS = call_api(url)
  tier_data = json.loads(REQUEST_TIERS.text)
  tier_df = json_normalize(tier_data)
  tier_frames.append(tier_df)
divisions = pd.concat(tier_frames, ignore_index=True, sort=True)

In [12]:
divisions['division_tier'] = divisions['tier'] + '_' + divisions['rank'] 
divisions

,freshBlood,hotStreak,inactive,leagueId,leaguePoints,losses,queueType,rank,summonerId,summonerName,tier,veteran,wins,division_tier
0,False,False,False,bba31c20-0f0a-11ea-9d15-0292af83ade6,94,5,RANKED_TFT,I,yuST92qI7G8ZiMWVBzHJ6e2xONnD6Rc1R-AfC3IU-FEsK8xY,Thesmokingpun,IRON,False,0,IRON_I
1,False,False,False,38ef74e0-2049-11ea-af12-e2e7dba1695f,41,22,RANKED_TFT,I,cClZh8jH4gTqTlFoz_fYFO6OQwLvWEvyJaCx515J_g6Ml3GX,TwitchDrDescend,IRON,False,0,IRON_I
2,False,False,False,3e06330f-c4f3-4a08-a497-e81f6df10588,13,9,RANKED_TFT,I,8VN_E8UqdztjMGB_OU5taldKe8rYdKqbQeQNJij7aa8Bx8BS,Iskoog,IRON,False,0,IRON_I
3,False,False,False,f4694230-0bb0-11ea-b9fc-eeb4098ebaa0,11,6,RANKED_TFT,I,Ild9nWTxkyF8XWYHIbSKkO4l_HuWKnc-oCBhnMwm2txdJVdu,paybackisthename,IRON,False,0,IRON_I
4,False,False,False,13e67db0-19e1-11ea-8ce1-0256c6254f55,30,5,RANKED_TFT,I,cID8eSPpX2QVsydzTIdzHtXD1GBIjf_kTsl3zIXIctjPOspl,Dasheng Tian,IRON,False,2,IRON_I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,False,False,False,633cfc20-14a2-11ea-83dd-969dda9e1b7b,51,17,RANKED_TFT,I,jvA0o1G8Wm0EBeadAqZKiBcr_CpuDo5Ajmd4K8REoOzb6vRg,LuckyOt,BRONZE,False,2,BRONZE_I
406,False,False,False,c3f1cab0-0f8d-11ea-894e-1ad76ae081af,64,22,RANKED_TFT,I,JTceaF-VQcXWeAT7n8dLFlRuce2sBxk1Zs7R6N-UUouer0bq,Iron4Lif3,BRONZE,False,5,BRONZE_I
407,False,False,False,a61a9f40-0dac-11ea-894e-1ad76ae081af,26,23,RANKED_TFT,I,NINZYVxeUFffmuaso63Wt5VjeeXbUWCQDUoZfg6H-PqFte8J,R0SH,BRONZE,False,1,BRONZE_I
408,False,False,False,e9333520-0d30-11ea-9c9a-c2363367468d,0,29,RANKED_TFT,I,h6ncE9IrOSGZOgjWZSzF-KQCjL5xP7kbdfOyKwRKD2fQFDae,SirChosen1,BRONZE,False,4,BRONZE_I


In [18]:
divisions

,freshBlood,hotStreak,inactive,leagueId,leaguePoints,losses,queueType,rank,summonerId,summonerName,tier,veteran,wins,division_tier
0,False,False,False,bba31c20-0f0a-11ea-9d15-0292af83ade6,94,5,RANKED_TFT,I,yuST92qI7G8ZiMWVBzHJ6e2xONnD6Rc1R-AfC3IU-FEsK8xY,Thesmokingpun,IRON,False,0,IRON_I
1,False,False,False,38ef74e0-2049-11ea-af12-e2e7dba1695f,41,22,RANKED_TFT,I,cClZh8jH4gTqTlFoz_fYFO6OQwLvWEvyJaCx515J_g6Ml3GX,TwitchDrDescend,IRON,False,0,IRON_I
2,False,False,False,3e06330f-c4f3-4a08-a497-e81f6df10588,13,9,RANKED_TFT,I,8VN_E8UqdztjMGB_OU5taldKe8rYdKqbQeQNJij7aa8Bx8BS,Iskoog,IRON,False,0,IRON_I
3,False,False,False,f4694230-0bb0-11ea-b9fc-eeb4098ebaa0,11,6,RANKED_TFT,I,Ild9nWTxkyF8XWYHIbSKkO4l_HuWKnc-oCBhnMwm2txdJVdu,paybackisthename,IRON,False,0,IRON_I
4,False,False,False,13e67db0-19e1-11ea-8ce1-0256c6254f55,30,5,RANKED_TFT,I,cID8eSPpX2QVsydzTIdzHtXD1GBIjf_kTsl3zIXIctjPOspl,Dasheng Tian,IRON,False,2,IRON_I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,False,False,False,633cfc20-14a2-11ea-83dd-969dda9e1b7b,51,17,RANKED_TFT,I,jvA0o1G8Wm0EBeadAqZKiBcr_CpuDo5Ajmd4K8REoOzb6vRg,LuckyOt,BRONZE,False,2,BRONZE_I
406,False,False,False,c3f1cab0-0f8d-11ea-894e-1ad76ae081af,64,22,RANKED_TFT,I,JTceaF-VQcXWeAT7n8dLFlRuce2sBxk1Zs7R6N-UUouer0bq,Iron4Lif3,BRONZE,False,5,BRONZE_I
407,False,False,False,a61a9f40-0dac-11ea-894e-1ad76ae081af,26,23,RANKED_TFT,I,NINZYVxeUFffmuaso63Wt5VjeeXbUWCQDUoZfg6H-PqFte8J,R0SH,BRONZE,False,1,BRONZE_I
408,False,False,False,e9333520-0d30-11ea-9c9a-c2363367468d,0,29,RANKED_TFT,I,h6ncE9IrOSGZOgjWZSzF-KQCjL5xP7kbdfOyKwRKD2fQFDae,SirChosen1,BRONZE,False,4,BRONZE_I


In [21]:
# Get summoner name from divisions data frame / keeping it at 5 for now for testing
summoner_names = divisions.loc[ :, 'summonerName' ]

In [22]:
summoner_names

0         Thesmokingpun
1       TwitchDrDescend
2                Iskoog
3      paybackisthename
4          Dasheng Tian
             ...       
405             LuckyOt
406           Iron4Lif3
407                R0SH
408          SirChosen1
409             Voltoxs
Name: summonerName, Length: 410, dtype: object

In [24]:
summoner_frames = []

for summoner in summoner_names:
  summoner_url = API_BASE + 'tft/summoner/v1/summoners/by-name/' + summoner + '?api_key=' + KEY
  REQUEST_NAME = call_api(summoner_url)
  summoner_data = json.loads(REQUEST_NAME.text)
  summoner_df = json_normalize(summoner_data)
  summoner_frames.append(summoner_df)
join_summoners = pd.concat(summoner_frames, ignore_index=True, sort=True)

In [25]:
join_summoners = join_summoners.merge(divisions[['summonerName','division_tier']],left_on='name', right_on = 'summonerName')

In [26]:
join_summoners.head()

,accountId,id,name,profileIconId,puuid,revisionDate,summonerLevel,summonerName,division_tier
0,zI283lT_AmT46PYA4ryvDu6_pCEQWelYXsDEFnkVuY46eK...,yuST92qI7G8ZiMWVBzHJ6e2xONnD6Rc1R-AfC3IU-FEsK8xY,Thesmokingpun,4080,Ki1KO62ygcycSVm6LoqHuxrL2CMzEF-bU2uMKrms9pPUog...,1577783118000,174,Thesmokingpun,IRON_I
1,HoX-vMezy19MFPStaMIWJkSJo7TqzKUiiZcdRLQJNiLNfj...,cClZh8jH4gTqTlFoz_fYFO6OQwLvWEvyJaCx515J_g6Ml3GX,TwitchDrDescend,1437,Qo8MH3V2TR24SNwj15PHwdLnmls1zV2TshFuMzLrwoK30-...,1583335989000,27,TwitchDrDescend,IRON_I
2,JpUFYXoEICpGy_pc_bpIF834PYUkAxCZU_hNvNPs9Zedy4...,8VN_E8UqdztjMGB_OU5taldKe8rYdKqbQeQNJij7aa8Bx8BS,Iskoog,3791,rOLjK2bI7u1Vgf0UW6N57SMSE26R_-Q5sNj5CCYLNPhzCu...,1583315599000,38,Iskoog,IRON_I
3,L1syM0T2vG7fFnguMokQk2F_XmRwby3YtItKvx-naNEIFp...,Ild9nWTxkyF8XWYHIbSKkO4l_HuWKnc-oCBhnMwm2txdJVdu,paybackisthename,4088,PhT-sWyWBFP7e9ks7BgJ-jJ4tbI1Pu9g_YqQWf4v-pWQJb...,1583110821000,147,paybackisthename,IRON_I
4,A70edyG8emCOgQguKKxXH_UCI1Uw2ypDg7Ogai9khr4sHj...,cID8eSPpX2QVsydzTIdzHtXD1GBIjf_kTsl3zIXIctjPOspl,Dasheng Tian,3546,NaxTv335VOuWqE47Re-qtlj6IFINYFe9mO0pH_lYndCdlF...,1580409701000,13,Dasheng Tian,IRON_I


In [27]:
# Get puuid from summoners data frame
summoner_puuid = join_summoners.loc[ : , 'puuid' ]

In [28]:
summoner_puuid

0      Ki1KO62ygcycSVm6LoqHuxrL2CMzEF-bU2uMKrms9pPUog...
1      Qo8MH3V2TR24SNwj15PHwdLnmls1zV2TshFuMzLrwoK30-...
2      rOLjK2bI7u1Vgf0UW6N57SMSE26R_-Q5sNj5CCYLNPhzCu...
3      PhT-sWyWBFP7e9ks7BgJ-jJ4tbI1Pu9g_YqQWf4v-pWQJb...
4      NaxTv335VOuWqE47Re-qtlj6IFINYFe9mO0pH_lYndCdlF...
                             ...                        
405    tYxDg1v3Ctr5p-pvKRMFHme6dWW0NGS0HOLiG5gFwPdaka...
406    5P8tcyCXgYbUK69IG68PTkrBgKnRsAWnhlNZ4ivkwMHcFU...
407    aHURIN5A47xjJR_UP_NqQrDJU35qtr9cOoQou_VIEmmJdT...
408    sAuYVLuRperHidY_aBhAsvYOuV-b0a063_LIFgmyhVVFdV...
409    ifQwfNwkgYiIOzkyJVLVpW_e7vzBfx38BLZLxM_Vp2RtG5...
Name: puuid, Length: 410, dtype: object

In [29]:
game_id_frames = []
match_count = '1'

for puuid in summoner_puuid:
  match_url = API_BASE_REGION + '/tft/match/v1/matches/by-puuid/' + puuid + '/ids?count=' + match_count + '&api_key=' + KEY
  REQUEST_MATCH = call_api(match_url)
  match_data = json.loads(REQUEST_MATCH.text)
  match_df = pd.Series(match_data)
  game_id_frames.append(match_df)
game_id_data = pd.concat(game_id_frames, ignore_index=True, sort=True)

In [30]:
# need to convert to series
game_id_list = game_id_data.loc[:]

In [31]:
game_id_list

0      NA1_3220227240
1      NA1_3314634985
2      NA1_3302506161
3      NA1_3313864862
4      NA1_3281128705
            ...      
405    NA1_3289842831
406    NA1_3229759227
407    NA1_3246149807
408    NA1_3309019666
409    NA1_3304881732
Length: 410, dtype: object

In [32]:
match_info_frames = []
match_count = '2'

for game_id in game_id_list:
  result_url = API_BASE_REGION + 'tft/match/v1/matches/' + game_id + '?api_key=' + KEY
  REQUEST_RESULT = call_api(result_url)
  result_data = json.loads(REQUEST_RESULT.text)
  result_df = json_normalize(result_data)
  match_info_frames.append(result_df)
match_info_data = pd.concat(match_info_frames, ignore_index=True, sort=True)

In [33]:
match_info_data

,info.game_datetime,info.game_length,info.game_version,info.participants,info.queue_id,info.tft_set_number,metadata.data_version,metadata.match_id,metadata.participants
0,1574977840977,2028.678955,Version 9.23.299.3089 (Nov 25 2019/17:12:47) [...,[{'companion': {'content_ID': 'b5cab758-7194-4...,1100,2,3,NA1_3220227240,[WwAaD7WefPc_IIK298IpwEWE-FYn2s3kpLM9kSdsaxGTA...
1,1583335985615,2226.008789,Version 10.5.311.166 (Feb 28 2020/20:49:49) [P...,[{'companion': {'content_ID': '14c5e00e-14af-4...,1090,2,4,NA1_3314634985,[5ZHcC8Aal0rksbL6IJNvYiwX6zqG_nqtecktef1FC1LVE...
2,1582241300313,1823.497559,Version 10.4.308.9400 (Feb 14 2020/17:25:03) [...,[{'companion': {'content_ID': 'c4508a6f-123d-4...,1100,2,4,NA1_3302506161,[2K2P4x46UgHOhGTI32nxjcT_PahCXUisXEowi6vbatO0O...
3,1583269972996,1979.711426,Version 10.4.308.9400 (Feb 14 2020/17:25:03) [...,[{'companion': {'content_ID': 'd07d0b76-cd0d-4...,1100,2,4,NA1_3313864862,[plODai2a7lalVCepZefOt1MIkaJzkKAHCzb3hqbXVJa0B...
4,1580409699694,2011.223267,Version 10.2.305.4739 (Jan 21 2020/15:48:11) [...,[{'companion': {'content_ID': '5897ad9f-4665-4...,1100,2,4,NA1_3281128705,[NaxTv335VOuWqE47Re-qtlj6IFINYFe9mO0pH_lYndCdl...
...,...,...,...,...,...,...,...,...,...
405,1581143037035,2068.713867,Version 10.3.307.1028 (Feb 03 2020/16:35:06) [...,[{'companion': {'content_ID': '5897ad9f-4665-4...,1090,2,4,NA1_3289842831,[jnp-MpGMFqSe7TuoJrdQ07TyFICpSkG9wDFXzotdEbqjC...
406,1575900321140,2340.039307,Version 9.23.299.3089 (Nov 25 2019/17:12:47) [...,[{'companion': {'content_ID': '5897ad9f-4665-4...,1100,2,3,NA1_3229759227,[eb6n2vasGyvKI-RcgTvI2nag2mt-lHJVWe4zxSLwvkBSK...
407,1577404418256,1987.621582,Version 9.24.300.6382 (Dec 06 2019/17:15:05) [...,[{'companion': {'content_ID': '069301c3-099a-4...,1100,2,4,NA1_3246149807,[b_vlgq2digW7VlBIkWyi9tvNloc-di_S0v_ypl636ox_D...
408,1582847525479,2082.392334,Version 10.4.308.9400 (Feb 14 2020/17:25:03) [...,[{'companion': {'content_ID': 'f56ecac7-ed16-4...,1100,2,4,NA1_3309019666,[sAuYVLuRperHidY_aBhAsvYOuV-b0a063_LIFgmyhVVFd...


In [21]:
match_info_data.columns

Index(['info.game_datetime', 'info.game_length', 'info.game_version',
       'info.participants', 'info.queue_id', 'info.tft_set_number',
       'metadata.data_version', 'metadata.match_id', 'metadata.participants'],
      dtype='object')

In [159]:
pd.options.display.max_seq_items = 2000

In [22]:
match_info_data['info.participants'][2]

[{'companion': {'content_ID': 'c4508a6f-123d-443f-84e7-e40ea526cd7f',
   'skin_ID': 10,
   'species': 'PetPenguKnight'},
  'gold_left': 2,
  'last_round': 27,
  'level': 6,
  'placement': 7,
  'players_eliminated': 0,
  'puuid': '2K2P4x46UgHOhGTI32nxjcT_PahCXUisXEowi6vbatO0OKKH77lGgjPgUHu-wwS_AkcEuNsqbXlJtA',
  'time_eliminated': 1490.4603271484375,
  'total_damage_to_players': 42,
  'traits': [{'name': 'Berserker',
    'num_units': 1,
    'style': 0,
    'tier_current': 0,
    'tier_total': 2},
   {'name': 'Inferno',
    'num_units': 3,
    'style': 1,
    'tier_current': 1,
    'tier_total': 3},
   {'name': 'Light',
    'num_units': 1,
    'style': 0,
    'tier_current': 0,
    'tier_total': 3},
   {'name': 'Set2_Ranger',
    'num_units': 2,
    'style': 1,
    'tier_current': 1,
    'tier_total': 3},
   {'name': 'Shadow',
    'num_units': 3,
    'style': 1,
    'tier_current': 1,
    'tier_total': 2},
   {'name': 'Summoner',
    'num_units': 3,
    'style': 1,
    'tier_current': 1,

In [35]:
#df = pd.Series(data={match_info_data['info.participants'][0]})
# dct = {'A':5, 'B':4, 'C':-1}
# df['min_count'] = df['category'].map(dct)
# df = df.query('count>min_count')

participants_df = []
for num in range(len(match_info_data['info.participants'])):
    
    data = match_info_data['info.participants'][num]
    data_df = pd.DataFrame.from_dict(data)
    participants_df.append(data_df)
    game_df = pd.concat(participants_df, ignore_index=True, sort=True)


game_df


,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
0,{'content_ID': 'b5cab758-7194-412a-8370-908b11...,3,33,8,4,0,WwAaD7WefPc_IIK298IpwEWE-FYn2s3kpLM9kSdsaxGTAr...,1897.483276,126,"[{'name': 'Berserker', 'num_units': 3, 'style'...","[{'character_id': 'TFT2_Volibear', 'items': []..."
1,{'content_ID': '3fd8f774-fbf9-4ffd-aed5-79f1a8...,2,31,6,6,0,T56QuzTRqNPZjCw_HV07qsaLk0jsSn3EahJgBlbo76J0qz...,1769.266968,81,"[{'name': 'Crystal', 'num_units': 1, 'style': ...","[{'character_id': 'TFT2_Skarner', 'items': [6]..."
2,{'content_ID': 'b1299bd0-b2e3-4b60-bdf6-a66d38...,1,34,7,3,0,Ki1KO62ygcycSVm6LoqHuxrL2CMzEF-bU2uMKrms9pPUog...,1957.186768,108,"[{'name': 'Electric', 'num_units': 1, 'style':...","[{'character_id': 'TFT2_Veigar', 'items': [34,..."
3,{'content_ID': '2e631ec1-f5ea-4e21-b57b-3b8d92...,11,30,7,7,0,5aiIUH1lI9kbERkF4IUXt7h8u_ibETAXHzF-6MUxmOLTuZ...,1692.093506,74,"[{'name': 'Berserker', 'num_units': 6, 'style'...","[{'character_id': 'TFT2_Jax', 'items': [14, 29..."
4,{'content_ID': '2a705022-688c-4405-bee4-3598b6...,20,33,7,5,2,vON1Whx0BlQ7lk4SMkXcwORVZjVmgX1VI7b5lqT2RTu_uv...,1891.783081,86,"[{'name': 'Alchemist', 'num_units': 1, 'style'...","[{'character_id': 'TFT2_Twitch', 'items': [16,..."
...,...,...,...,...,...,...,...,...,...,...,...
3275,{'content_ID': '4b799da4-88d3-41db-ad4a-b937f8...,2,33,8,3,0,GkdwQjI6I0CVgE7DS4_A9zmVoXq8DiAofZm2YUyYUUPZoJ...,1809.952026,94,"[{'name': 'Inferno', 'num_units': 1, 'style': ...","[{'character_id': 'TFT2_Taliyah', 'items': [],..."
3276,{'content_ID': 'b5dae839-0b3a-4ebb-827d-651de9...,4,30,6,6,0,wnyzN6RFighACdwrPYnL-ujmEEuQt4uTNAHhhSeYfLswX4...,1653.206177,93,"[{'name': 'Berserker', 'num_units': 1, 'style'...","[{'character_id': 'TFT2_Taliyah', 'items': [],..."
3277,{'content_ID': 'c159fe8e-a7c6-4256-9397-d888e9...,3,31,7,5,1,ifQwfNwkgYiIOzkyJVLVpW_e7vzBfx38BLZLxM_Vp2RtG5...,1716.478271,101,"[{'name': 'Avatar', 'num_units': 1, 'style': 4...","[{'character_id': 'TFT2_Ornn', 'items': [], 'n..."
3278,{'content_ID': 'ee7592da-2140-4576-97dd-c6f374...,43,33,7,4,0,dMSD9sXfdoqwf2JgRb6Gy7E2Qyc8QtGX5gTgYD3dCim2kx...,1806.946777,128,"[{'name': 'Druid', 'num_units': 2, 'style': 3,...","[{'character_id': 'TFT2_Maokai', 'items': [], ..."


In [ ]:
range(len(game_df['units']))

In [36]:
game_df

,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
0,{'content_ID': 'b5cab758-7194-412a-8370-908b11...,3,33,8,4,0,WwAaD7WefPc_IIK298IpwEWE-FYn2s3kpLM9kSdsaxGTAr...,1897.483276,126,"[{'name': 'Berserker', 'num_units': 3, 'style'...","[{'character_id': 'TFT2_Volibear', 'items': []..."
1,{'content_ID': '3fd8f774-fbf9-4ffd-aed5-79f1a8...,2,31,6,6,0,T56QuzTRqNPZjCw_HV07qsaLk0jsSn3EahJgBlbo76J0qz...,1769.266968,81,"[{'name': 'Crystal', 'num_units': 1, 'style': ...","[{'character_id': 'TFT2_Skarner', 'items': [6]..."
2,{'content_ID': 'b1299bd0-b2e3-4b60-bdf6-a66d38...,1,34,7,3,0,Ki1KO62ygcycSVm6LoqHuxrL2CMzEF-bU2uMKrms9pPUog...,1957.186768,108,"[{'name': 'Electric', 'num_units': 1, 'style':...","[{'character_id': 'TFT2_Veigar', 'items': [34,..."
3,{'content_ID': '2e631ec1-f5ea-4e21-b57b-3b8d92...,11,30,7,7,0,5aiIUH1lI9kbERkF4IUXt7h8u_ibETAXHzF-6MUxmOLTuZ...,1692.093506,74,"[{'name': 'Berserker', 'num_units': 6, 'style'...","[{'character_id': 'TFT2_Jax', 'items': [14, 29..."
4,{'content_ID': '2a705022-688c-4405-bee4-3598b6...,20,33,7,5,2,vON1Whx0BlQ7lk4SMkXcwORVZjVmgX1VI7b5lqT2RTu_uv...,1891.783081,86,"[{'name': 'Alchemist', 'num_units': 1, 'style'...","[{'character_id': 'TFT2_Twitch', 'items': [16,..."
...,...,...,...,...,...,...,...,...,...,...,...
3275,{'content_ID': '4b799da4-88d3-41db-ad4a-b937f8...,2,33,8,3,0,GkdwQjI6I0CVgE7DS4_A9zmVoXq8DiAofZm2YUyYUUPZoJ...,1809.952026,94,"[{'name': 'Inferno', 'num_units': 1, 'style': ...","[{'character_id': 'TFT2_Taliyah', 'items': [],..."
3276,{'content_ID': 'b5dae839-0b3a-4ebb-827d-651de9...,4,30,6,6,0,wnyzN6RFighACdwrPYnL-ujmEEuQt4uTNAHhhSeYfLswX4...,1653.206177,93,"[{'name': 'Berserker', 'num_units': 1, 'style'...","[{'character_id': 'TFT2_Taliyah', 'items': [],..."
3277,{'content_ID': 'c159fe8e-a7c6-4256-9397-d888e9...,3,31,7,5,1,ifQwfNwkgYiIOzkyJVLVpW_e7vzBfx38BLZLxM_Vp2RtG5...,1716.478271,101,"[{'name': 'Avatar', 'num_units': 1, 'style': 4...","[{'character_id': 'TFT2_Ornn', 'items': [], 'n..."
3278,{'content_ID': 'ee7592da-2140-4576-97dd-c6f374...,43,33,7,4,0,dMSD9sXfdoqwf2JgRb6Gy7E2Qyc8QtGX5gTgYD3dCim2kx...,1806.946777,128,"[{'name': 'Druid', 'num_units': 2, 'style': 3,...","[{'character_id': 'TFT2_Maokai', 'items': [], ..."


In [24]:
traits_data = game_df['traits']
traits_df = pd.DataFrame.from_dict(traits_data)
traits_df['traits'][0]

[{'name': 'Berserker',
  'num_units': 3,
  'style': 1,
  'tier_current': 1,
  'tier_total': 2},
 {'name': 'Electric',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3},
 {'name': 'Metal',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3},
 {'name': 'Mountain',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 1},
 {'name': 'Poison',
  'num_units': 3,
  'style': 3,
  'tier_current': 1,
  'tier_total': 1},
 {'name': 'Predator',
  'num_units': 3,
  'style': 3,
  'tier_current': 1,
  'tier_total': 1},
 {'name': 'Set2_Glacial',
  'num_units': 4,
  'style': 2,
  'tier_current': 2,
  'tier_total': 3},
 {'name': 'Set2_Ranger',
  'num_units': 1,
  'style': 0,
  'tier_current': 0,
  'tier_total': 3},
 {'name': 'Warden',
  'num_units': 2,
  'style': 1,
  'tier_current': 1,
  'tier_total': 3}]

In [25]:
units_data = game_df['units']
units_df = pd.DataFrame.from_dict(units_data)
units_df['units'][1]

[{'character_id': 'TFT2_Skarner',
  'items': [6],
  'name': 'Skarner',
  'rarity': 1,
  'tier': 2},
 {'character_id': 'TFT2_KogMaw',
  'items': [17, 10201],
  'name': 'KogMaw',
  'rarity': 0,
  'tier': 2},
 {'character_id': 'TFT2_Ivern',
  'items': [33],
  'name': 'Ivern',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT2_Maokai',
  'items': [69],
  'name': 'Maokai',
  'rarity': 0,
  'tier': 3},
 {'character_id': 'TFT2_RekSai',
  'items': [45],
  'name': 'RekSai',
  'rarity': 1,
  'tier': 2},
 {'character_id': 'TFT2_Nocturne',
  'items': [10201, 12, 2],
  'name': 'Nocturne',
  'rarity': 2,
  'tier': 3}]

In [37]:
# join the unit name with tier value in a list separated by comma
game_df['units_name'] = [','.join([unit['name']+'_'+str(unit['tier']) for unit in units]) for units in game_df['units']]

# create dummy columns of the list of 'unit_tier' per player and add it to original df
units_df = game_df['units_name'].str.get_dummies(sep = ',')

In [38]:
# join the trait name with set tier value in a list separated by comma
game_df['trait_name'] = [','.join([trait['name']+'_'+str(trait['tier_current']) for trait in traits]) for traits in game_df['traits']]

# create dummy columns of the list of 'unit_tier' per player and add it to original df
traits_df = game_df['trait_name'].str.get_dummies(sep = ',')

In [39]:
# join the dummy columns with original game_df
game_df = game_df.join(units_df)
game_df = game_df.join(traits_df)


In [41]:
game_df = game_df.merge(join_summoners[['puuid','summonerName','division_tier']],left_on='puuid', right_on ='puuid')

In [48]:
game_df.to_csv('./datasets/base_data_division.csv')

In [49]:
game_df = pd.read_csv('./datasets/base_data_division.csv')

In [47]:
game_df

,Unnamed: 0,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,...,Warden_3,Wind_0,Wind_1,Wind_2,Wind_3,Woodland_0,Woodland_1,Woodland_2,summonerName,division_tier
0,0,{'content_ID': 'b1299bd0-b2e3-4b60-bdf6-a66d38...,1,34,7,3,0,Ki1KO62ygcycSVm6LoqHuxrL2CMzEF-bU2uMKrms9pPUog...,1957.186768,108,...,0,0,0,0,0,1,0,0,Thesmokingpun,IRON_I
1,1,{'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...,1,31,7,6,0,Qo8MH3V2TR24SNwj15PHwdLnmls1zV2TshFuMzLrwoK30-...,1821.959229,116,...,0,0,0,0,0,0,0,0,TwitchDrDescend,IRON_I
2,2,{'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...,3,33,8,2,0,rOLjK2bI7u1Vgf0UW6N57SMSE26R_-Q5sNj5CCYLNPhzCu...,1815.009277,128,...,0,1,0,0,0,0,0,0,Iskoog,IRON_I
3,3,{'content_ID': 'eb78f626-c11a-480e-adeb-ed68af...,2,23,7,8,0,PhT-sWyWBFP7e9ks7BgJ-jJ4tbI1Pu9g_YqQWf4v-pWQJb...,1226.217285,21,...,0,1,0,0,0,0,0,0,paybackisthename,IRON_I
4,4,{'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...,0,27,7,6,0,NaxTv335VOuWqE47Re-qtlj6IFINYFe9mO0pH_lYndCdlF...,1502.661987,38,...,0,0,0,0,0,0,0,0,Dasheng Tian,IRON_I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,405,{'content_ID': '6570cfb2-8507-4cc7-a2e8-97e648...,0,37,8,3,0,tYxDg1v3Ctr5p-pvKRMFHme6dWW0NGS0HOLiG5gFwPdaka...,1992.891479,132,...,0,0,0,0,0,0,0,0,LuckyOt,BRONZE_I
406,406,{'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...,1,37,8,4,1,5P8tcyCXgYbUK69IG68PTkrBgKnRsAWnhlNZ4ivkwMHcFU...,2270.177002,155,...,0,0,0,0,0,0,0,0,Iron4Lif3,BRONZE_I
407,407,{'content_ID': '6921a827-770d-44f0-ba4f-9f171c...,8,33,7,3,0,aHURIN5A47xjJR_UP_NqQrDJU35qtr9cOoQou_VIEmmJdT...,1924.085327,111,...,0,0,0,0,0,0,0,0,R0SH,BRONZE_I
408,408,{'content_ID': 'f56ecac7-ed16-483d-8b9c-2bbced...,1,23,6,7,0,sAuYVLuRperHidY_aBhAsvYOuV-b0a063_LIFgmyhVVFdV...,1268.213257,25,...,0,0,0,0,0,0,0,0,SirChosen1,BRONZE_I


In [36]:
# RANK values? need to change dia -> bronze : 10 -> 1?